In [1]:
import os

config = {}
config["weights_file"] = os.getcwd() + '/model/weight'
config["patch_size"] = (64, 64, 64)  # switch to None to train on the whole image
config["patch_gap"] = 16
config["batch_size"] = 10
config["kfold"] = 5

config["input_shape"] = (1, None, None, None)
config["depth"] = 4 # depth of layers for V/Unet
config["n_base_filters"] = 32
config["pool_size"] = (2, 2, 2)  # pool size for the max pooling operations
config["deconvolution"] = True  # if False, will use upsampling instead of deconvolution

config["patience"] = 10  # learning rate will be reduced after this many epochs if the validation loss is not improving
config["early_stop"] = 10  # training will be stopped after this many epochs without the validation loss improving
config["initial_learning_rate"] = 0.00001
config["learning_rate_drop"] = 0.5  # factor by which the learning rate will be reduced
config["n_epochs"] = 1

In [2]:
from model.data import *
from model.model import *

d = Data()
d.load_data(config["patch_size"])

# prepare data for training
train_num, valid_num = d.prekfold(config["patch_size"], config["patch_gap"], config["batch_size"], config["kfold"])

Using TensorFlow backend.


In [ ]:

model = unet_model_3d(input_shape=config["input_shape"],
                              pool_size=config["pool_size"],
                              initial_learning_rate=config["initial_learning_rate"],
                              deconvolution=config["deconvolution"],
                              depth=config["depth"],
                              n_base_filters=config["n_base_filters"])

# model.load_weights(os.getcwd() + '/model/weight/weights-02-0.01.hdf5')
model.load_weights(os.getcwd() + '/model/weight/weights-01-0.01-0425.hdf5')


In [ ]:
merge_image = reconstruct_from_patches(image, indices, shape[-3:])
merge_target = reconstruct_from_patches(target, indices, shape[-3:])
merge_result = reconstruct_from_patches(result, indices, shape[-3:])
d.show_image([merge_image[None, None, :], merge_target[None, None, :], merge_result[None, None, :]>0.01])

In [ ]:
sel = 13
print(image[sel].shape, target[sel].shape, result[sel].shape)
d.show_image([image[sel][None, :], target[sel][None, :], result[sel]>0])
print("target:", np.mean(target[sel]))
print("result:", np.mean(result[sel]))
print(dice_coefficient(target[sel], result[sel]))

print(np.unique(result[sel]))